In [201]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from collections import Counter

In [189]:
source_info = pd.read_csv('../data/SourceInfo.csv')
caa_c0=pd.read_csv("../data/caa_nrc_cluster_0.csv")
caa_c1=pd.read_csv("../data/caa_nrc_cluster_1.csv")
caa_c2=pd.read_csv("../data/caa_nrc_cluster_2.csv")
covid_c0=pd.read_csv("../data/covid_cluster_0.csv")
covid_c1=pd.read_csv("../data/covid_cluster_1.csv")
covid_c2=pd.read_csv("../data/covid_cluster_2.csv")
original_parsed_data = pd.read_csv("../data/parsed_cleaned.csv")
strong_words_data = pd.read_csv("../data/StrongWordsAnalysis.csv")

In [190]:
articles_copy = caa_c0
articles_copy = articles_copy.append([caa_c1,caa_c2,covid_c0,covid_c1,covid_c2])

In [207]:
articles = []
for i in original_parsed_data.index:
    source=original_parsed_data['Source'].iloc[i]
    title=original_parsed_data['Title'].iloc[i]
    topic=original_parsed_data['Topic Name'].iloc[i]
    text=original_parsed_data['Cleaned_text'].iloc[i]
    source_reliability=source_info.loc[source_info['Source']==source,'SourceScore'].iloc[0]
    strong_words_score = strong_words_data.loc[strong_words_data['Title']==title,'strongP'].iloc[0]
    neutrality_score=articles_copy['neutrality'].iloc[i]
    articles.append([source,title,topic,text,source_reliability,strong_words_score,neutrality_score])

In [209]:
articles_data = pd.DataFrame(articles,columns=["Source","Title","Topic Name","Cleaned Text","Source Score","Strong Words Score","Neutrality Score"])
articles_data.head()

,Source,Title,Topic Name,Cleaned Text,Source Score,Strong Words Score,Neutrality Score
0,The Times of India,"PM Modi, Amit Shah must engage in dialogue ove...",caa-nrc,download time news app latest news subscribe s...,1.0,0.047619,0.273091
1,The Times of India,"Trump to discuss CAA, NRC issues with Modi dur...",caa-nrc,united state president trump prime minister fi...,1.0,0.157895,0.188507
2,The Times of India,"Sonia Gandhi: PM Modi, Amit Shah misled people...",caa-nrc,download time news app latest news subscribe s...,1.0,0.094488,0.164222
3,The Times of India,"I told PM Modi to withdraw CAA, NRC and NPR: M...",caa-nrc,download time news app latest news subscribe s...,1.0,0.000000,0.224548
4,The Times of India,PM Modi's visit to Bangladesh: Simmering tensi...,caa-nrc,prime minister file photo download time news a...,1.0,0.069231,0.271837


In [210]:
articles_data['Reliability Score']=""
for i in articles_data.index:
    score = (1/3)*articles_data['Source Score'][i]+(1/3)*articles_data['Neutrality Score'][i]-(1/3)*articles_data['Strong Words Score'][i]
    articles_data['Reliability Score'][i] = round(score,3)

C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [211]:
scores=np.reshape(np.asarray(articles_data['Reliability Score']),(-1,1))
scaler = MinMaxScaler()
scaler.fit(scores)
scores=scaler.transform(scores)
print(scores)

articles_data['Reliability Score']=scores

[[0.93712575]
 [0.84131737]
 [0.86077844]
 ...
 [0.49251497]
 [0.5239521 ]
 [0.47005988]]


In [212]:
articles_data['Reliability Label']=""
for i in articles_data.index:

    if articles_data['Reliability Score'][i]>=0.60:
        articles_data['Reliability Label'][i]='Reliable'
    else:
        articles_data['Reliability Label'][i]='Not Reliable'

C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [219]:
reliable_articles=articles_data[articles_data['Reliability Label']=='Reliable']
sources_count=Counter(reliable_articles['Source'])
sources_count

Counter({'The Times of India': 67,
         'Indian Express': 93,
         'The Hindu': 62,
         'News18': 18,
         'NDTV': 7,
         'DDNews': 25,
         'The Quint': 6,
         'Times Now': 5,
         'The Print': 5,
         'Hindustan Times': 58,
         'India Today': 15,
         'Organizer': 1,
         'Deccan Chronicle': 1,
         'Republic TV': 8,
         'The Statesman': 50,
         'Reuters': 48,
         'The Telegraph': 11,
         'Zee News': 21,
         'Independent': 1,
         'OpIndia': 1})

Counter({'The Times of India': 67,
         'Indian Express': 93,
         'The Hindu': 62,
         'News18': 18,
         'NDTV': 7,
         'DDNews': 25,
         'The Quint': 6,
         'Times Now': 5,
         'The Print': 5,
         'Hindustan Times': 58,
         'India Today': 15,
         'Organizer': 1,
         'Deccan Chronicle': 1,
         'Republic TV': 8,
         'The Statesman': 50,
         'Reuters': 48,
         'The Telegraph': 11,
         'Zee News': 21,
         'Independent': 1,
         'OpIndia': 1})